In [ ]:
# install dependencies: 
#!pip install pyyaml==5.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
import torch
#assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# exit(0)  # After installation, you need to "restart runtime" in Colab. This line can also restart runtime

# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from matplotlib import pyplot as plt
from visualizer_cc_dsg import Visualizer_cc
from detectron2.utils.visualizer import Visualizer


os.environ['CUDA_VISIBLE_DEVICES']="4"



In [2]:
from detectron2.data.datasets import register_coco_instances

for d in ["train"]:
    register_coco_instances(f"AMI_{d}", {}, f"Data_testrun_AMI/syn/{d}.json", f"Data_testrun_AMI/syn")
for d in ["test"]:
    register_coco_instances(f"AMI_{d}", {}, f"Data_testrun_AMI/{d}/{d}.json", f"Data_testrun_AMI/{d}")    

In [8]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# cfg.DATASETS.TRAIN = ("EDS_PP_train",)
# cfg.DATASETS.TEST = ("EDS_PP_test")
# cfg.DATALOADER.NUM_WORKERS = 2
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
# cfg.SOLVER.IMS_PER_BATCH = 2
# cfg.SOLVER.BASE_LR = 0.00025
# cfg.SOLVER.MAX_ITER = 1000*20
# cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4

from detectron2.engine import DefaultTrainer

cfg = get_cfg()

# cfg.MODEL.DEVICE='cpu'
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ('AMI_train') 
cfg.DATASETS.TEST = (['AMI_test'])#("EDS_PP_test")
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")
#cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final_0621.pth")
# Let training initialize from model zoo
#cfg.MODEL.ANCHOR_GENERATOR.SIZES=[[32], [64], [128], [256], [512]]
# cfg.MODEL.ANCHOR_GENERATOR.SIZES=[ [16], [32], [64], [128], [256]] # (setting 1)
#cfg.MODEL.ANCHOR_GENERATOR.SIZES=[  [16], [32], [64], [96], [128]] # (setting 2)
cfg.MODEL.ANCHOR_GENERATOR.SIZES=[[8], [16], [32], [64], [128]] # (setting 3)
#cfg.MODEL.ANCHOR_GENERATOR.ANGLES=[[-90],[0],[90]]
cfg.SOLVER.IMS_PER_BATCH =20
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 3000 #6000*4   # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.CHECKPOINT_PERIOD= 200
cfg.SOLVER.STEPS = []
cfg.SOLVER.STEPS = [16000]        # do not decay learning rate
#cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3   # set a custom testing threshold
cfg.TEST.EVAL_PERIOD = 200
cfg.TEST.DETECTIONS_PER_IMAGE=20
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1 #5-2  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
#trainer = DefaultTrainer(cfg) 
cfg.MODEL.ROI_HEADS.NMS_THRESH = 0.05
print(cfg)

CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 2
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ['AMI_test']
  TRAIN: AMI_train
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: [[-90, 0, 90]]
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES: [[8], [16], [32], [64], [128]]
  BACKBONE:
    FREEZE_AT: 2
    NAME: build_resnet_fpn_backbone
  DEVICE: cuda
  FPN:
    FUSE_TYPE: sum
    IN_FEATURES: ['res2', 'res3', 'res4', 'res

In [6]:
import random
# from google.colab.patches import cv2_imshow
#from detectron2.utils.visualizer import Visualizer
#cfg.MODEL.DEVICE='cpu'
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final_AMI_20K_20230126.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.98   # set a custom testing threshold
cfg.TEST.DETECTIONS_PER_IMAGE=500
predictor = DefaultPredictor(cfg)

folder = "Data_testrun_AMI/" + "unlabel_10lots" 

dataset_dicts = DatasetCatalog.get(f"AMI_test") #(path + "train")
for d in dataset_dicts: # random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN), scale=1.0)
    v = visualizer.draw_dataset_dict(d)
    file = d["file_name"].split('/')[-1].rsplit(".",1)[0]
    print(file)
    path = folder + "/" +file+".jpg"
    plt.imsave(path,v.get_image()[:, :, ::-1])
#    plt.figure(figsize = (30,40))
#    plt.imshow(vis.get_image()[:, :, ::-1])
    
    outputs = predictor(img)
    text_msg="model_final_AMI_20K_20230126.pth, created by Data Science Group, EMAIL: PDL-Data-Science-Group@wdc.com"
    visualizer = Visualizer_cc(img[:, :, ::-1], metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN), scale=1.0)
    v = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"), text_msg)
#    plt.imshow(v.get_image()[:, :, ::-1])
    
 #   file = file
#    print(file)
    new_file = file+'_pred'+".jpg"
    path = folder + "/" + new_file 
    plt.imsave(path,v.get_image()[:, :, ::-1])
#     cv2_imshow(vis.get_image()[:, :, ::-1])
    
    
#Ours    
#     v = Visualizer_cc(im[:, :, ::-1], metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN), scale=1.0)
#     out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
#     plt.figure(figsize = (30,40))
#     plt.imshow(out.get_image()[:, :, ::-1])

WARNING [01/30 18:36:05 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[01/30 18:36:05 d2.data.datasets.coco]: Loaded 114 images in COCO format from Data_testrun_AMI/test/test.json
DEL018968_SLOT09_DIF2_K3PEP.EX1_AMI
DEL018970_SLOT10_DIF1_K3PEP.EX1_AMI
DE0018178_SLOT15_DIF2_K3PEP.EX1_AMI
DE0018224_SLOT23_DIF2_K3PEP.EX1_AMI
DEL019190_SLOT17_DIF5_K3PEP.EX1_AMI
DEL018970_SLOT10_DIF4_K3PEP.EX1_AMI
DEL019189_SLOT04_DIF1_K3PEP.EX1_AMI
DEL018971_SLOT14_DIF1_K3PEP.EX1_AMI
DEL018068_SLOT05_DIF5_K3 PEP.EX1_A6YU6S0_AMI
DEL018815_SLOT15_DIF5_K3PEP.EX1_AMI
DE0018178_SLOT15_DIF4_K3PEP.EX1_AMI
DEL019188_SLOT08_DIF3_K3PEP.EX1_AMI
DEL018068_SLOT05_DIF3_K3 PEP.EX1_A6YU6S0_AMI
DE0018195_SLOT17_DIF1_K3PEP.EX1_AMI
DE0018178_SLOT10_DIF4_K3PEP.EX1_AMI
DEL019190_SLOT17_DIF4_K3PEP.EX1_AMI
DEL018906_SLOT07_DIF1_K3PEP.EX1_AMI
DEL019193_SLOT19_DIF2_K3PEP.EX1_AMI
DEL019188_SLOT08_DIF5_K3PEP.EX1_AMI
DEL018968_SLOT10_DIF2_K3PEP.EX1_AMI
DEL018815_SLOT

In [10]:
import os
import glob, sys


cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final_AMI_20K_20230126.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.98   # set a custom testing threshold
cfg.TEST.DETECTIONS_PER_IMAGE=500
predictor = DefaultPredictor(cfg)
# filelist = glob.glob(r"Data_testrun_AMI/unlabel_10lots/*.jpg")
filelist = glob.glob(r"Data_testrun_AMI/unlabeled_/*.jpg")

# dirname="unlabeled_"
# image_in_dir='Data_testrun_AMI/' + dirname + '/*.jpg'
# out_path=os.path.split(image_in_dir)[0]+"_out"  
# print(out_path)
# try:
#     os.mkdir(out_path)
# except:
#     print("already a folder")

# unlabeled_file_dir = out_path
unlabeled_file_dir='Data_testrun_AMI/unlabeled_'

txt_msg="CNN model version:model_final_AMI_20K_20230126, created by Data Science Group, Email: PDL-Data-Science-Group@wdc.com"
filelist=os.listdir(unlabeled_file_dir)


#filelist=["DEL019846_SLOT01_DIF1_K3 PEP.EX1_TYY03SRC_AMI.jpg"]
for d in filelist: # random.sample(dataset_dicts, 3):
#    print(d)
    img = cv2.imread(os.path.join(unlabeled_file_dir,d))
    visualizer = Visualizer_cc(img[:, :, ::-1], metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN), scale=1.0)
#    v = visualizer.draw_dataset_dict(d)
#    plt.figure(figsize = (30,40))
#    plt.imshow(vis.get_image()[:, :, ::-1])
    
    outputs = predictor(img)
 #   print(outputs)
#     visualizer = Visualizer_cc(img[:, :, ::-1], metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN), scale=1.2)
    v = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"),txt_msg)
#    plt.imshow(v.get_image()[:, :, ::-1])
    
    
    file = d.split('/')[-1]
    file = file.rsplit(".",1)[0]
#    print(file)
    print(len(outputs["instances"].to("cpu").pred_classes.numpy().tolist()))
    new_file = file+'_pred'+".jpg"
    
    path = unlabeled_file_dir + "/" + new_file 
#    print(path)
    plt.imsave(path,v.get_image()[:, :, ::-1])

[211.34084 149.90865 240.60403 179.84785]
[238.65622 397.7209  267.31958 426.628  ]
[554.74066 373.8911  582.8118  402.14667]
[619.133   185.51492 647.6057  214.01562]
4
[238.60986 398.17288 267.66766 427.36655]
[617.9679  187.22493 646.7738  215.97197]
2


TypeError: 'NoneType' object is not subscriptable

# Code for inference without labels and generation of sample and defect information

In [70]:
##### output boxes ######    
def XY_centroid(outputs,scale):
    instances = outputs["instances"]
    Class = outputs["instances"].to("cpu").pred_classes
    pred_class_probs = outputs["instances"].to("cpu").scores.numpy()
    index = (Class==0).nonzero(as_tuple=True)[0].numpy() ### select a specific class
    a = Class[index].cpu()
    a= a.numpy()
    #    print(a)
    #  output prediction data    
    Box = outputs["instances"].to("cpu").pred_boxes
    b=Box.tensor.cpu()[index]
    b_st = b[b[:,0].sort()[1]] 
    b_st=b_st.numpy()
    
    pred_scores=[]
    XYs=[]
    xs= []
    for _ , row in enumerate(b_st):
        x = row
        X = -((x[0] + x[2])/2-614)*scale*10**3 - 6188 #um #((x[0] + x[2])/2)*scale wrt wafer center
        Y = -((x[1] + x[3])/2-614)*scale*10**3 + 30 #um #((x[1] + x[3])/2)*scale   wrt wafer center
        X_Glob = math.floor( (X + 6188)/12377 ) + 11
        Y_Glob = math.floor( (X + 2228.5)/4517 ) + 32
        XYs.append([X, Y])
        xs.append(x)
#         print(x)

    for prob in pred_class_probs:
        pred_scores.append(prob)
        
    
    return XYs, xs, pred_scores

In [102]:
print(pred_scores)
print(xs[0])

[0.9994418]
[582.8506  119.29022 612.1635  148.88356]


In [105]:
for i, x in enumerate(xs):
    print(x[0])

582.8506


In [113]:
import os
import glob, sys
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler

# cfg.MODEL.DEVICE='cuda'
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final_AMI_20K_20230126.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.98   # set a custom testing threshold
cfg.TEST.DETECTIONS_PER_IMAGE=500
predictor = DefaultPredictor(cfg)
# filelist = glob.glob(r"Data_testrun_AMI/unlabel_10lots/*.jpg")

dirname="unlabeled_1"#sys.argv[1]
image_in_dir='Data_testrun_AMI/' + dirname + '/*.jpg'

filelist = glob.glob(image_in_dir)

out_path=os.path.split(image_in_dir)[0]+"_out"  
print(out_path)
try:
    os.mkdir(out_path)
except:
    print("already a folder")

unlabeled_file_outdir = out_path

txt_msg="CNN model version:model_final_AMI_20K_20230126, created by Data Science Group, Email: PDL-Data-Science-Group@wdc.com"

data_list=[]
# filelist=['Data_testrun_AMI/unlabeled_1/AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_DE0018195.00C_s17_DIF1_OK.jpg']

for d in filelist: # random.sample(dataset_dicts, 3):
#     print(d)
    XY=[]
    img = cv2.imread(d) #os.path.join(unlabeled_file_dir,d))
    visualizer = Visualizer_cc(img[:, :, ::-1], metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN), scale=1.0)
    #    v = visualizer.draw_dataset_dict(d)
    #    plt.imshow(vis.get_image()[:, :, ::-1])

    outputs = predictor(img)
    #   print(outputs)
    #     visualizer = Visualizer_cc(img[:, :, ::-1], metadata=MetadataCatalog.get(cfg.DATASETS.TRAIN), scale=1.2)
    v = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"),txt_msg)
    
    file = d.split('/')[-1]
    file = file.rsplit(".",1)[0]       
    column_values = file.split("_")[1:-1]
#     transform(column_values[3])to_datetime_format
    column_names = ['Product ID', 'Process Step','Inspection time','Lot No','Slot No','DIF']
    data_dict = {name: value for name, value in zip(column_names, column_values)}
    
    XY, xs, pred_scores = XY_centroid(outputs, 0.2717) #0.326) ### needs scale
    print(d) 
    
    for i, (r1, x, score) in enumerate(zip(XY, xs, pred_scores)):
        # Create a new dictionary with the column values
        new_data_dict = data_dict.copy()
        
        # Add the calculated column for the current 2D array
        new_data_dict['XY'] = r1
        new_data_dict['Confidence Score'] = round(score, 4)

#     plt.figure(figsize = (30,40))
#         plt.imshow(v.get_image()[int(x[1])-2:int(x[3])+15, int(x[0])-2:int(x[2])+18, ::-1])
        new_file = file+'_bbox'+str(i)+".jpg"
        path = unlabeled_file_outdir + "/" + new_file
        plt.imsave(path,v.get_image()[int(x[1])-2:int(x[3])+15, int(x[0])-2:int(x[2])+18, ::-1])
#         new_data_dict = data_dict.copy()
        new_data_dict['Fig'] = new_file
        data_list.append(new_data_dict)
    
    
    
#     # Loop through each 2D array in XY
#     for i, r1 in enumerate(XY):
#         # Create a new dictionary with the column values
#         new_data_dict = data_dict.copy()
        
#         # Add the calculated column for the current 2D array
#         new_data_dict['XY'] = r1
        
#         # Add the row to the list of dictionaries
#         data_list.append(new_data_dict)
    
#     for i, score in enumerate(pred_scores):
#         # Create a new dictionary with the column values
# #         new_data_dict = data_dict.copy()
#         # Add the calculated column for the current 2D array
#         new_data_dict['Confidence Score'] = score #round(score, 4)
# #         print(score)
#         # Add the row to the list of dictionaries
#         data_list.append(new_data_dict)
          
#     for i, x in enumerate(xs):    
# #     plt.figure(figsize = (30,40))
#         plt.imshow(v.get_image()[int(x[1])-2:int(x[3])+15, int(x[0])-2:int(x[2])+18, ::-1])
#         new_file = file+'_bbox'+str(i)+".jpg"
#         path = unlabeled_file_outdir + "/" + new_file
#         plt.imsave(path,v.get_image()[int(x[1])-2:int(x[3])+15, int(x[0])-2:int(x[2])+18, ::-1])
# #         new_data_dict = data_dict.copy()
#         new_data_dict['Fig'] = new_file
#         data_list.append(new_data_dict)
    
#     if len(XY) == 1:
#         XY_concat = XY[0]
#     else:
#         XY_concat = np.concatenate(XY, axis=1)
#     data_dict['X','Y'] = XY 
#     data_dict['Y'] = XY[1]
#     print(data_dict)


#     data_list.append(data_dict)

# Convert dictionary to pandas dataframe
column_order = column_names + ['Confidence Score'] + ['XY'] + ['Fig']

df = pd.DataFrame(data_list, columns = column_order)
print(df)

# Step 1: Select the first 5 columns and drop duplicate rows
cols_to_match = column_names[:-1] ### remove DIF
unique_combinations = df[cols_to_match].drop_duplicates()

# Step 2: Iterate over each unique combination and cluster the corresponding 2D arrays
clustered_data = []
index_to_label = {}
for _, row in unique_combinations.iterrows():
    match_rows =  df[(df[cols_to_match] == row).all(axis=1)] ## find all matching rows
#     X = np.vstack(match_rows['XY'].values) 
#     X = np.vstack(np.array([np.reshape(row, (-1, 2)) for row in match_rows['XY'].values]))
#     X = np.concatenate(match_rows['XY'].values).reshape(-1, 2)
#     scaler = StandardScaler()
#     X = np.ceil(scaler.fit_transform(X))
    X = np.vstack(match_rows['XY'].values).astype(float) # Convert values to float and stack 1D arrays
    X_norms = np.linalg.norm(X, axis=1)
    max_norm_idx = np.argmax(X_norms)
    X = X / X_norms[max_norm_idx] # Normalize by the largest norm
#     print(row)
#     print(X)
#     X = np.ceil(X)
#     n = len(match_rows['XY'].values)
#     if n == 1:
#         n_clusters = 1 
#     else:
#         n_clusters = int(2)
    dbscan = DBSCAN(eps=0.05, min_samples=1).fit(X)
#     kmeans = KMeans(n_clusters=n_clusters, n_init=1, random_state=0,tol=1e-10).fit(X)
#     cluster_labels = kmeans.predict(X)
#     # create the MiniBatchKMeans model
#     kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=n,tol=1e-10)
#     # fit the model to the data
#     kmeans.fit(X)
#     cluster_labels = kmeans.labels_

    match_rows.loc[:, 'Defect ID'] = dbscan.labels_
    clustered_data.append(match_rows)
#     for i, label in enumerate(dbscan.labels_):
#             index_to_label[match_rows.index[i]] = label
#     row['Defect ID'] = dbscan.labels_
#     clustered_data.append(row)

# Step 3: Convert the clustered data to a dataframe and merge it with the original dataframe
# clustered_df = pd.DataFrame(clustered_data)
clustered_df = pd.concat(clustered_data)

grouped_df = df.merge(clustered_df[['Defect ID']], left_index=True, right_index=True, how='inner')
# grouped_df = df.merge(clustered_df, on=cols_to_match, how='left')

# Step 4: Add the cluster labels to the grouped dataframe
# grouped_df['Defect ID'] = grouped_df.index.map(index_to_label)
# grouped_df = grouped_df.join(df['Defect ID'], on=df.index)

grouped_df['X'] = [xy[0] for xy in grouped_df['XY']]
grouped_df['Y'] = [xy[1] for xy in grouped_df['XY']]
grouped_df = grouped_df.drop('XY', axis=1)

column_order = column_names + ['Confidence Score'] + ["Defect ID"] + ['X'] + ['Y'] + ['Fig']
grouped_df_reorder = grouped_df[column_order]

# time_formatted = []
# for t in grouped_df['Inspection time']:
#     dt = datetime.strptime(t, '%Y-%m-%d-%H-%M-%S')
#     time_formatted.append(dt.strftime('%Y/%m/%d %H:%M:%S'))
# grouped_df['Inspection time'] = time_formatted

print(clustered_df)
print(grouped_df)
#     new_file = file+'_pred'+".jpg"

#     path = unlabeled_file_outdir + "/" + new_file 
#     #    print(path)
#     plt.imsave(path,v.get_image()[:, :, ::-1])

Data_testrun_AMI/unlabeled_1_out
already a folder
Data_testrun_AMI/unlabeled_1/AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_DE0018195.00C_s17_DIF1_OK.jpg
Data_testrun_AMI/unlabeled_1/AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_DEL018906.00C_s07_DIF3_OK.jpg
Data_testrun_AMI/unlabeled_1/AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_DEL018968.00C_s05_DIF2_OK.jpg
Data_testrun_AMI/unlabeled_1/AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_DEL018968.00C_s07_DIF3_OK.jpg
Data_testrun_AMI/unlabeled_1/AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_DE0018224.00C_s23_DIF1_OK.jpg
Data_testrun_AMI/unlabeled_1/AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_DEL018906.00C_s07_DIF4_OK.jpg
Data_testrun_AMI/unlabeled_1/AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_DE0018178.00C_s10_DIF1_OK.jpg
Data_testrun_AMI/unlabeled_1/AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_DEL018968.00C_s05_DIF4_OK.jpg
Data_testrun_AMI/unlabeled_1/AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_DEL018906.00C_s07_DIF5_OK.jpg
Data_testrun_A

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

   Product ID Process Step      Inspection time         Lot No Slot No   DIF  \
0      A6YU6S   K3 PEP.EX1  2023-01-01-12-34-56  DE0018195.00C     s17  DIF1   
1      A6YU6S   K3 PEP.EX1  2023-01-01-12-34-56  DEL018906.00C     s07  DIF3   
14     A6YU6S   K3 PEP.EX1  2023-01-01-12-34-56  DEL018906.00C     s07  DIF4   
21     A6YU6S   K3 PEP.EX1  2023-01-01-12-34-56  DEL018906.00C     s07  DIF5   
43     A6YU6S   K3 PEP.EX1  2023-01-01-12-34-56  DEL018906.00C     s07  DIF2   
69     A6YU6S   K3 PEP.EX1  2023-01-01-12-34-56  DEL018906.00C     s07  DIF1   
2      A6YU6S   K3 PEP.EX1  2023-01-01-12-34-56  DEL018968.00C     s05  DIF2   
3      A6YU6S   K3 PEP.EX1  2023-01-01-12-34-56  DEL018968.00C     s05  DIF2   
4      A6YU6S   K3 PEP.EX1  2023-01-01-12-34-56  DEL018968.00C     s05  DIF2   
5      A6YU6S   K3 PEP.EX1  2023-01-01-12-34-56  DEL018968.00C     s05  DIF2   
6      A6YU6S   K3 PEP.EX1  2023-01-01-12-34-56  DEL018968.00C     s05  DIF2   
7      A6YU6S   K3 PEP.EX1  2023-01-01-1

In [106]:
grouped_df

,Product ID,Process Step,Inspection time,Lot No,Slot No,DIF,Confidence Score,Fig,Defect ID,X,Y
0,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018195.00C,s17,DIF1,0.999637,AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0,282.527152,206.637122
1,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018195.00C,s17,DIF1,0.999637,AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0,282.527152,206.637122
2,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018195.00C,s17,DIF1,0.999637,AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0,282.527152,206.637122
3,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018906.00C,s07,DIF3,0.999119,AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0,283.514106,225.046650
4,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018906.00C,s07,DIF3,0.999119,AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0,283.514106,225.046650
5,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018906.00C,s07,DIF3,0.999119,AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0,283.514106,225.046650
6,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018968.00C,s05,DIF2,NaN,NaN,0,56.306044,229.529190
7,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018968.00C,s05,DIF2,NaN,NaN,1,75.984265,208.958280
8,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018968.00C,s05,DIF2,NaN,NaN,2,87.657021,173.529557
9,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018968.00C,s05,DIF2,NaN,NaN,3,126.964773,45.272872


In [114]:
grouped_df_reorder.to_csv(unlabeled_file_outdir + "/" +"final_output_test.csv")

In [91]:
clustered_df

,Product ID,Process Step,Inspection time,Lot No,Slot No,DIF,Confidence Score,XY,Fig,Defect ID
0,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018195.00C,s17,DIF1,0.9996,"[282.5271518554688, 206.63712158203126]",AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0
1,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018195.00C,s17,DIF1,0.9996,"[282.5271518554688, 206.63712158203126]",AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0
2,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018195.00C,s17,DIF1,0.9996,"[282.5271518554688, 206.63712158203126]",AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0
3,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018906.00C,s07,DIF3,0.9991,"[283.5141057128906, 225.04665039062502]",AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0
4,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018906.00C,s07,DIF3,0.9991,"[283.5141057128906, 225.04665039062502]",AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0
5,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018906.00C,s07,DIF3,0.9991,"[283.5141057128906, 225.04665039062502]",AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0
42,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018906.00C,s07,DIF4,0.9998,"[283.0283688964844, 224.7648426513672]",AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0
43,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018906.00C,s07,DIF4,0.9998,"[283.0283688964844, 224.7648426513672]",AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0
44,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018906.00C,s07,DIF4,0.9998,"[283.0283688964844, 224.7648426513672]",AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0
63,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018906.00C,s07,DIF5,0.9997,"[283.1112020263672, 224.75475463867187]",AMIimg_A6YU6S_K3 PEP.EX1_2023-01-01-12-34-56_D...,0


In [78]:
# sorted_grouped_df = grouped_df.sort_values(by=cols_to_match)
sorted_grouped_df

,Product ID,Process Step,Inspection time,Lot No,Slot No,DIF,Defect ID,X,Y
15,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018178.00C,s10,DIF1,0,166.072088,166.369356
22,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018178.00C,s10,DIF5,0,166.072775,166.234501
45,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018178.00C,s10,DIF4,0,166.053544,166.236720
46,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018178.00C,s10,DIF3,0,166.117604,166.624780
66,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018178.00C,s10,DIF2,0,166.127393,166.283031
0,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018195.00C,s17,DIF1,0,282.527152,206.637122
13,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018224.00C,s23,DIF1,0,194.372963,44.399667
33,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018224.00C,s23,DIF5,0,194.374316,44.388151
44,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018224.00C,s23,DIF4,0,195.110343,44.359872
47,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018224.00C,s23,DIF2,0,195.200379,44.024456


In [325]:
dbscan.labels_

array([0, 1, 2, 0, 2, 0, 1, 2, 0, 2, 0, 2])

In [354]:
row

Product ID                      A6YU6S
Process Step                K3 PEP.EX1
Inspection time    2023-01-01-12-34-56
Lot No                   DEL018970.00C
Slot No                            s23
Name: 23, dtype: object

In [356]:
row = row.drop('Defect ID')

In [357]:
grouped_df[(grouped_df[cols_to_match] == row).all(axis=1)]

,Product ID,Process Step,Inspection time,Lot No,Slot No,DIF,Defect ID,X,Y
23,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF3,0,89.858307,269.433429
24,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF3,1,168.990946,22.094929
25,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF3,2,186.805203,298.504614
34,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF1,0,89.766709,269.341184
35,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF1,2,186.762185,298.417105
40,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF2,0,89.795709,269.359012
41,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF2,1,169.312251,22.322285
42,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF2,2,186.513725,298.566634
64,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF4,0,89.969792,269.148855
65,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF4,2,186.771477,298.325696


In [330]:
match_rows

,Product ID,Process Step,Inspection time,Lot No,Slot No,DIF,XY,Defect ID
23,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF3,"[89.85830651855468, 269.4334287109375]",0
24,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF3,"[168.99094641113282, 22.094929061889648]",1
25,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF3,"[186.80520288085938, 298.50461401367187]",2
34,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF1,"[89.76670855712891, 269.34118408203125]",0
35,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF1,"[186.7621845703125, 298.41710498046876]",2
40,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF2,"[89.79570910644532, 269.35901220703124]",0
41,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF2,"[169.31225061035155, 22.322284912109374]",1
42,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF2,"[186.51372497558594, 298.56663439941406]",2
64,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF4,"[89.96979199218751, 269.1488552246094]",0
65,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,DIF4,"[186.77147668457033, 298.32569604492187]",2


In [338]:
clustered_df

,Product ID,Process Step,Inspection time,Lot No,Slot No,Defect ID
0,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018195.00C,s17,[-1]
1,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018906.00C,s07,"[0, 0, 0, 0, 0]"
2,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018968.00C,s05,"[0, 0, 0, 1, 2, 2, 0, 1, 1, 2, 2, 0, 1, 2, 2]"
8,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018968.00C,s07,"[0, 1, 1, 2, 2, 1, 3, 3, 4, 2, 2, 2, 0, 1, 3, ..."
13,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018224.00C,s23,"[0, 0, 0, 0, 0]"
15,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018178.00C,s10,"[0, 0, 0, 0, 0]"
23,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018970.00C,s23,"[0, 1, 2, 0, 2, 0, 1, 2, 0, 2, 0, 2]"


In [341]:
index_to_label

{0: -1,
 1: 0,
 14: 0,
 21: 0,
 43: 0,
 69: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 1,
 6: 2,
 7: 2,
 16: 0,
 17: 1,
 18: 1,
 19: 2,
 20: 2,
 36: 0,
 37: 1,
 38: 2,
 39: 2,
 8: 0,
 9: 1,
 10: 1,
 11: 2,
 12: 2,
 26: 1,
 27: 3,
 28: 3,
 29: 4,
 30: 2,
 31: 2,
 32: 2,
 48: 0,
 49: 1,
 50: 3,
 51: 3,
 52: 1,
 53: 4,
 54: 4,
 55: 2,
 56: 1,
 57: 1,
 58: 3,
 59: 3,
 60: 1,
 61: 4,
 62: 2,
 63: 2,
 13: 0,
 33: 0,
 44: 0,
 47: 0,
 70: 0,
 15: 0,
 22: 0,
 45: 0,
 46: 0,
 66: 0,
 23: 0,
 24: 1,
 25: 2,
 34: 0,
 35: 2,
 40: 0,
 41: 1,
 42: 2,
 64: 0,
 65: 2,
 67: 0,
 68: 2}

In [315]:
df

,Product ID,Process Step,Inspection time,Lot No,Slot No,DIF,XY
0,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DE0018195.00C,s17,DIF1,"[282.5271518554688, 206.63712158203126]"
1,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018906.00C,s07,DIF3,"[283.5141057128906, 225.04665039062502]"
2,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018968.00C,s05,DIF2,"[56.306043884277344, 229.5291901855469]"
3,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018968.00C,s05,DIF2,"[75.9842646484375, 208.95827978515626]"
4,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018968.00C,s05,DIF2,"[87.65702056884766, 173.52955725097658]"
5,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018968.00C,s05,DIF2,"[126.96477301025391, 45.27287194824219]"
6,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018968.00C,s05,DIF2,"[177.9419783935547, 82.95618572998048]"
7,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018968.00C,s05,DIF2,"[189.34294494628907, 62.169109313964846]"
8,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018968.00C,s07,DIF3,"[20.504367073059083, 167.47726696777343]"
9,A6YU6S,K3 PEP.EX1,2023-01-01-12-34-56,DEL018968.00C,s07,DIF3,"[66.31732202148437, 261.16034301757816]"


In [331]:
clustered_df

,0
0,Product ID Process Step Inspection time...
1,Product ID Process Step Inspection tim...
2,Product ID Process Step Inspection tim...
3,Product ID Process Step Inspection tim...
4,Product ID Process Step Inspection tim...
5,Product ID Process Step Inspection tim...
6,Product ID Process Step Inspection tim...


In [302]:
match_rows.index[:]

Int64Index([23, 24, 25, 34, 35, 40, 41, 42, 64, 65, 67, 68], dtype='int64')

In [308]:
# X = np.vstack(np.array([np.reshape(row, (-1, 2)) for row in match_rows['XY'].values]))
X

array([[0.25517922, 0.76513586],
       [0.47989974, 0.06274508],
       [0.53048859, 0.84769208],
       [0.2549191 , 0.7648739 ],
       [0.53036642, 0.84744357],
       [0.25500146, 0.76492453],
       [0.48081218, 0.06339073],
       [0.52966085, 0.8478682 ],
       [0.25549582, 0.76432772],
       [0.53039281, 0.84718399],
       [0.2553875 , 0.76465975],
       [0.53042485, 0.84769699]])

In [298]:
index_to_label

{0: -1,
 1: 0,
 14: 0,
 21: 0,
 43: 0,
 69: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 1,
 6: 2,
 7: 2,
 16: 0,
 17: 1,
 18: 1,
 19: 2,
 20: 2,
 36: 0,
 37: 1,
 38: 2,
 39: 2,
 8: 0,
 9: 1,
 10: 1,
 11: 2,
 12: 2,
 26: 1,
 27: 3,
 28: 3,
 29: 4,
 30: 2,
 31: 2,
 32: 2,
 48: 0,
 49: 1,
 50: 3,
 51: 3,
 52: 1,
 53: 4,
 54: 4,
 55: 2,
 56: 1,
 57: 1,
 58: 3,
 59: 3,
 60: 1,
 61: 4,
 62: 2,
 63: 2,
 13: 0,
 33: 0,
 44: 0,
 47: 0,
 70: 0,
 15: 0,
 22: 0,
 45: 0,
 46: 0,
 66: 0,
 23: 0,
 24: 1,
 25: 2,
 34: 0,
 35: 2,
 40: 0,
 41: 1,
 42: 2,
 64: 0,
 65: 2,
 67: 0,
 68: 2}

In [299]:
dbscan.labels_

array([0, 1, 2, 0, 2, 0, 1, 2, 0, 2, 0, 2])

In [333]:
clustered_df

,0
0,Product ID Process Step Inspection time...
1,Product ID Process Step Inspection tim...
2,Product ID Process Step Inspection tim...
3,Product ID Process Step Inspection tim...
4,Product ID Process Step Inspection tim...
5,Product ID Process Step Inspection tim...
6,Product ID Process Step Inspection tim...


In [121]:
match_rows['XY'].values

array([list([866.647705078125, 633.856201171875])], dtype=object)

In [81]:
XY_concat = np.concatenate(XY, axis=0)

In [218]:
grouped_df['X'] = [xy[0] for xy in grouped_df['XY']]

In [219]:
grouped_df['X']

0     282.527152
1     283.514106
2     283.028369
3     283.111202
4     283.119539
5     283.045361
6      56.306044
7      75.984265
8      87.657021
9     126.964773
10    177.941978
11    189.342945
12     56.217331
13    120.245987
14    126.792789
15    177.534021
16    188.063479
17     55.917710
18    119.816720
19    189.165241
20    189.024685
21     20.504367
22     66.317322
23    135.751074
24    246.790039
25    294.091357
26     64.129726
27     94.712610
28    130.168473
29    217.141190
         ...    
41     64.264640
42     92.296840
43     93.533447
44    130.293956
45    135.248434
46    217.004654
47    247.586454
48    294.643910
49    194.372963
50    194.374316
51    195.110343
52    195.200379
53    194.787308
54    166.072088
55    166.072775
56    166.053544
57    166.117604
58    166.127393
59     89.858307
60    168.990946
61    186.805203
62     89.766709
63    186.762185
64     89.795709
65    169.312251
66    186.513725
67     89.969792
68    186.7714

In [220]:
grouped_df['XY']

0      [282.5271518554688, 206.63712158203126]
1      [283.5141057128906, 225.04665039062502]
2       [283.0283688964844, 224.7648426513672]
3      [283.1112020263672, 224.75475463867187]
4         [283.1195390625, 224.69060522460939]
5          [283.045361328125, 224.87951171875]
6      [56.306043884277344, 229.5291901855469]
7       [75.9842646484375, 208.95827978515626]
8      [87.65702056884766, 173.52955725097658]
9      [126.96477301025391, 45.27287194824219]
10      [177.9419783935547, 82.95618572998048]
11    [189.34294494628907, 62.169109313964846]
12      [56.2173310546875, 229.63810888671875]
13     [120.24598742675782, 35.45887216186524]
14    [126.79278930664063, 46.015574523925785]
15     [177.53402075195314, 83.54810534667969]
16       [188.063478515625, 62.78040408325196]
17     [55.917710113525395, 229.6542059326172]
18     [119.81671960449219, 35.37581518554688]
19     [189.16524072265625, 62.05753430175781]
20     [189.02468505859375, 62.97362335205079]
21    [20.504

In [227]:
XY_centroid(outputs,0.326)

[582.8506  119.29022 612.1635  148.88356]


[[194.78730810546875, 43.712324035644535]]

In [ ]:
import glob

filelist = glob.glob(r"Data_testrun_AMI/unlabeled/*.jpg")
print(filelist)

In [15]:
print(outputs)
print(len(outputs["instances"].to("cpu").pred_classes.numpy().tolist()))

{'instances': Instances(num_instances=0, image_height=1024, image_width=1024, fields=[pred_boxes: Boxes(tensor([], device='cuda:0', size=(0, 4))), scores: tensor([], device='cuda:0'), pred_classes: tensor([], device='cuda:0', dtype=torch.int64)])}
0


In [13]:
import pickle

# Assume your config object is called "cfg"
with open("AMI_config.pkl", "wb") as f:
    pickle.dump(cfg, f)